# Gravity inversion

We solve gravity potential, $\phi$, which takes the form of a Poisson equation:

$$
\nabla^2 \phi = -4\pi \gamma \rho
$$

where the vertical component of gravitational acceleration is its derivative, $\mathbf{g}_z = \nabla \phi$

All boundaries are Dirichlet and set to zero,

$$
\phi_{0,j,k} = \phi_{n,j,k} = \phi_{i,0,k} = \phi_{i,n,k} = \phi_{i,j,0} = \phi_{i,j,n} = 0
$$

Current approaches assign various layers to the crust and alter the density or geometry of layers to fit the gravity anomaly. This is a practical way to proceed if the underlying crustal architecture is already pretty well constrained, but thsi approach breaks if we assume absolutely no prior information on the layout of crustal lithologies. Here, we derive the adjoint of the inverse gravity problem, which affords us to invert the density on every point on the mesh efficiently.

The Poisson appears in many geophysical applications

In [ ]:
%matplotlib inline

import numpy as np
import conduction
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from scipy.optimize import minimize
from conduction.inversion import InvObservation, InvPrior
from mpi4py import MPI
comm = MPI.COMM_WORLD

In [ ]:
def initialise(self):
    """
    Creates a constant matrix to be used for each iteration
    of the forward, tangent linear, and adjoint models.
    """
    # matrix
    self.mesh.update_properties(np.ones(mesh.nn), np.zeros(mesh.nn))
    for boundary in self.mesh.bc:
        self.mesh.boundary_condition(boundary, 0.0, flux=False)
    mat = self.mesh.construct_matrix()
    return


def rhs_solve_ad(self, phi, dphi, matrix=None, rhs=None):
    """
    Solve just the RHS
    """
    # No need to evaluate this if dphi == 0 everywhere
    adjoint = np.array(False)
    nphi = dphi.any()
    comm.Allreduce([nphi, MPI.BOOL], [adjoint, MPI.BOOL], op=MPI.LOR)
    if adjoint:
        if type(matrix) == type(None):
            matrix = self.mesh.construct_matrix(in_place=False)
        if type(rhs) == type(None):
            rhs = self.mesh.construct_rhs(in_place=False)
        
        rhs[:] = dphi

        gvec = self.mesh.gvec
        lvec = self.mesh.lvec

        # adjoint b vec
        db_ad = lvec
        
        # db = A^T * dphi
        self.ksp.solveTranspose(rhs._gdata, gvec)
        self.mesh.dm.globalToLocal(gvec, db_ad)
        return db_ad.array
    
def forward_model(x, self):
    rho = x[:]
    rhs = -4.0*np.pi*gamma*rho
    self.mesh.heat_sources[:] = rhs
    
    phi = self.linear_solve(self.mesh.mat)
    g = np.array(self.gradient(phi))
    
    cost = 0.0
    cost += self.objective_routine(gz=g[0]) # obs
    cost += self.objective_routine(rho=rho) # prior
    return cost

def tangent_linear(x, dx, self):
    rho = x[:]
    drho = dx[:]
    # drhs/drho = -4.0*np.pi*gamma
    rhs = -4.0*np.pi*gamma*rho
    drhs = -4.0*np.pi*gamma*drho
    
    self.mesh.heat_sources[:] = rhs
    phi = self.linear_solve(self.mesh.mat)
    g = np.array(self.gradient(phi))
    
    self.mesh.heat_sources[:] = drhs
    dphi = self.linear_solve(self.mesh.mat)
    dg = np.array(self.gradient(dphi))
    
    cost = 0.0
    cost += self.objective_routine(gz=g[0]) # obs
    cost += self.objective_routine(rho=rho) # prior
    dcdg   = self.objective_routine_ad(gz=g[0])
    dcdrho = self.objective_routine_ad(rho=rho)
    dc = np.sum(dcdg*dg[0]) + np.sum(dcdrho*drho)
    return cost, dc

def adjoint_model(x, self):
    rho = x[:]
    rhs = -4.0*np.pi*gamma*rho
    self.mesh.heat_sources[:] = rhs
    
    phi = self.linear_solve(self.mesh.mat)
    g = np.array(self.gradient(phi))
    
    cost = 0.0
    cost += self.objective_routine(gz=g[0]) # obs
    cost += self.objective_routine(rho=rho) # prior
    
    # Adjoint bit
    dg = np.zeros_like(g)
    dg[0] = self.objective_routine_ad(gz=g[0])
    drho  = self.objective_routine_ad(rho=rho)
    
    dphi = self.gradient_ad(dg, g)
    drhs = rhs_solve_ad(self, phi, -dphi, self.mesh.mat)

    drho += -4.0*np.pi*gamma*drhs
    return cost, drho

## 2D case

A cross section with 1D gravity observations on the top

In [ ]:
minX, maxX = 0.0, 1000.0
minY, maxY = -1000.0, 0.0
nx, ny = 100, 100

# initialise mesh
mesh = conduction.ConductionND([minX, minY], [maxX, maxY], (nx, ny))

# initialise inversion object
lith_index = np.ones(mesh.nn, dtype=np.int)
inv = conduction.InversionND(lith_index, mesh, solver='cg')
initialise(inv)

In [ ]:
# create test data - 1D line

yoffset = -200.0

x = np.linspace(minX, maxX, nx)
y = np.ones_like(x) * yoffset

anomaly = np.abs(np.sin(0.002*np.pi*x))
anomaly_std = np.ones_like(anomaly)*0.1
anomaly_coords = np.column_stack([x,y])

plt.plot(x, anomaly)

# add observation
gobs = InvObservation(anomaly, anomaly_std, anomaly_coords)
inv.add_observation(gz=gobs)

In [ ]:

# global variables
gamma = 6.67408e-11


# estimate starting density
x = np.ones(mesh.nn)*20
dx = 0.01*x

fm0 = forward_model(x, inv)
fm1 = forward_model(x + dx, inv)
tl = tangent_linear(x, dx, inv)
ad = adjoint_model(x, inv)

print("finite difference {}".format(fm1 - fm0))
print("tangent linear {}".format(tl[1]))
print("adjoint model {}".format(ad[1].dot(dx)))

In [ ]:
x0 = x.copy()

res = minimize(adjoint_model, x0, args=(inv), method='TNC', jac=True, options={'disp':True})
res

In [ ]:
# compute forward model
cost = forward_model(res.x, inv)
phi = inv.mesh.temperature[:]
g = np.array(inv.gradient(phi))

# interpolate gz at depth
xcoords = inv.mesh.grid_coords[0]
inv.ndinterp.values = g[0]
gz_top = inv.ndinterp(np.column_stack([xcoords, np.ones_like(xcoords)*(maxY - inv.grid_delta[-1])]))
gz_obs = inv.ndinterp(anomaly_coords)


gs = GridSpec(2, 2, wspace=0.05, hspace=0.05, width_ratios=[1, 0.05], height_ratios=[0.2,1])

fig = plt.figure(figsize=(8, 9))
ax1 = fig.add_subplot(gs[0,0], ylabel=r"$g_z$")
ax1.set_xticks([])
ax1.grid(True)
for spine in ["top", "bottom", "right", "left"]:
    ax1.spines[spine].set_visible(False)
ax1.plot(xcoords, anomaly, c='C0', linestyle='dashed', label='obs')
ax1.plot(xcoords, gz_obs, c='C0', label='sim')
ax1.plot(xcoords, gz_top, c='C1', label='top')
ax1.legend(bbox_to_anchor=(1,1), frameon=False)

ax2 = fig.add_subplot(gs[1,0])
im2 = ax2.imshow(res.x.reshape(mesh.n), origin='lower', extent=(minX, maxX, minY, maxY))

cbax = fig.add_subplot(gs[1,1])
fig.colorbar(im2, cax=cbax)

## 3D case

Crustal volume with a 2D surface of gravity observations on top

In [ ]:
minX, maxX = 0.0, 1000.0
minY, maxY = 0.0, 1000.0
minZ, maxZ = -1000.0, 0.0

nx, ny, nz = 50, 50, 100

# initialise mesh
mesh = conduction.ConductionND([minX, minY, minZ], [maxX, maxY, maxZ], (nx, ny, nz))

# initialise inversion object
lith_index = np.ones(mesh.nn, dtype=np.int)
inv = conduction.InversionND(lith_index, mesh, solver='cg')
initialise(inv)

In [ ]:
# create test data - 2D surface

zoffset = -200.0

xcoords = np.linspace(minX, maxX, nx)
ycoords = np.linspace(minY, maxY, ny)
xq, yq = np.meshgrid(xcoords, ycoords)
zq = np.ones_like(xq)*zoffset

anomaly_coords = np.column_stack([xq.ravel(), yq.ravel(), zq.ravel()])

anomaly_x = np.abs(np.sin(0.002*np.pi*anomaly_coords[:,0]))
anomaly_y = np.abs(np.sin(0.002*np.pi*anomaly_coords[:,1]))
anomaly = anomaly_x + anomaly_y
anomaly_std = np.ones_like(anomaly)*0.1

plt.imshow(anomaly.reshape(ny,nx))
plt.colorbar()

# add observation
gobs = InvObservation(anomaly, anomaly_std, anomaly_coords)
inv.add_observation(gz=gobs)

In [ ]:
# global variables
gamma = 6.67408e-11

# estimate starting density
x = np.ones(mesh.nn)*20
dx = 0.01*x

fm0 = forward_model(x, inv)
fm1 = forward_model(x + dx, inv)
tl = tangent_linear(x, dx, inv)
ad = adjoint_model(x, inv)

print("finite difference {}".format(fm1 - fm0))
print("tangent linear {}".format(tl[1]))
print("adjoint model {}".format(ad[1].dot(dx)))

In [ ]:
x0 = x.copy()

res = minimize(adjoint_model, x0, args=(inv), method='TNC', jac=True, options={'disp':True})
res

In [ ]:
cost = forward_model(res.x, inv)
phi = inv.mesh.temperature[:]
g = np.array(inv.gradient(phi))

# interpolate gz at depth
gz_obs = inv.interpolate(g[0], anomaly_coords)



fig = plt.figure(figsize=(9.5,3))

ax1 = fig.add_subplot(121)
im1 = ax1.imshow(gz_obs.reshape(ny,nx))
fig.colorbar(im1)

ax2 = fig.add_subplot(122)
im2 = ax2.imshow(anomaly.reshape(ny,nx))
fig.colorbar(im2)

## Prior information

So far we have assumed no prior knowledge of the density structure, but we can add constraints inferred by reference densities for the upper crust.

A prior covariance matrix $\mathbf{C}_p$ may be required to spatially correlate uncertainties in some cases. We offer three end-members:

1. Uncorrelated prior information ($\ell_2$-norm)
2. The whole mesh is correlated
3. Uncertainties are correlated within a single lithology

In [ ]:
from conduction.inversion import create_covariance_matrix
from conduction.inversion import gaussian_function

### 1. Uncorrelated

In [ ]:
prior = np.ones(mesh.nn)*res.x.mean()
sigma_p = 0.01*prior

rho_p = InvPrior(prior, sigma_p)
inv.add_prior(rho=rho_p)

In [ ]:
x = np.ones(mesh.nn)*20
dx = 0.01*x

fm0 = forward_model(x, inv)
fm1 = forward_model(x + dx, inv)
tl = tangent_linear(x, dx, inv)
ad = adjoint_model(x, inv)

print("finite difference {}".format(fm1 - fm0))
print("tangent linear {}".format(tl[1]))
print("adjoint model {}".format(ad[1].dot(dx)))

### 2. Correlated on the mesh

In [ ]:
L = 10.0
max_dist = L*4

prior = np.ones(mesh.nn)*res.x.mean()
sigma_p = 0.01*prior

cov = create_covariance_matrix(sigma_p, mesh.coords, max_dist, gaussian_function, L)
rho_p = InvPrior(prior, sigma_p, cov_mat=cov)
inv.add_prior(rho=rho_p)

In [ ]:
# estimate starting density
x = np.ones(mesh.nn)*20
dx = 0.01*x

fm0 = forward_model(x, inv)
fm1 = forward_model(x + dx, inv)
tl = tangent_linear(x, dx, inv)
ad = adjoint_model(x, inv)

print("finite difference {}".format(fm1 - fm0))
print("tangent linear {}".format(tl[1]))
print("adjoint model {}".format(ad[1].dot(dx)))

### 3. Correlated within lithologies